In [1]:
import dialogflow
from google.api_core.exceptions import InvalidArgument
import dialogflow_v2beta1
import os
from google.oauth2 import service_account
import pandas as pd
import numpy as np

In [37]:
from collections import defaultdict

In [2]:
import os
path = os.path.abspath(service_account.__file__)
path

'D:\\anaconda\\lib\\site-packages\\google\\oauth2\\service_account.py'

In [3]:
try:
    print(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) 
except:
    print("key not found")

key not found


# google client initialization

In [4]:
project_id = 'intent-prediction-ohdl'

In [5]:
credential_path = r"D:\ds_projects\dialogflow\key\intent-prediction-ohdl-3a0947516b7b.json"

In [6]:
credentials = service_account.Credentials.from_service_account_file(credential_path)

In [7]:
intents_client = dialogflow.IntentsClient(credentials=credentials)

In [8]:
parent = intents_client.project_agent_path(project_id)

# intent list availaible

In [22]:
intents = intents_client.list_intents(parent)

In [10]:
all_intent_list = []

In [40]:
name = intents_client.intent_path(project_id, '0c4456d7-24fc-44f3-90ad-a63214a7b5a9')

In [43]:
response = intents_client.get_intent(name, intent_view = dialogflow.enums.IntentView.INTENT_VIEW_FULL)

In [44]:
response.training_phrases

[name: "a03fd0fe-08f5-4192-92bc-15525da921b7"
type: EXAMPLE
parts {
  text: "Also update to me for Hardcopy of my policy as requested given"
}
, name: "b62e3400-4e61-4a9d-a30d-0493c27274fd"
type: EXAMPLE
parts {
  text: "As discussed please update pan number and send hard copy of policy documents on priority basis"
}
, name: "ddefe01a-bc1e-40f8-9355-db3d0940bedd"
type: EXAMPLE
parts {
  text: "To Request for updated Policy Pack So i request max life to please send me my Updated Policy Pack before renewal of my term Insurance and would also like to request a Call from Max Life Regarding Term Insurance"
}
, name: "1c95e3cc-fd8f-40d5-9049-39147faabaa0"
type: EXAMPLE
parts {
  text: "This is to inform you that when I amp apos m downloading my policy pack from your website it amp apos s showing me old details of the policy"
}
, name: "ceb9d908-0844-4f5e-8f7e-c923065a1fd8"
type: EXAMPLE
parts {
  text: "And send the Policy related documents on the above address"
}
, name: "ea26a462-5346-4b7f

In [48]:
def fetch_intents_training_phrases(service_account_file, project):

    dialogflow_entity_client = dialogflow.EntityTypesClient.from_service_account_file(service_account_file)
    parent = dialogflow_entity_client.project_agent_path(project)
    entities = list(dialogflow_entity_client.list_entity_types(parent))

    dialogflow_intents_client = dialogflow.IntentsClient.from_service_account_file(service_account_file)
    parent = dialogflow_intents_client.project_agent_path(project)
    intents = list(dialogflow_intents_client.list_intents(
    parent=parent,
    intent_view=dialogflow.enums.IntentView.INTENT_VIEW_FULL))

    entities_name_to_value = {
        'date-time': 'tomorrow afternoon',
        'date': 'tomorrow',
        'date-period': 'April',
        'time': '4:30 pm',
        'time-period': 'afternoon',
        'number': 'one',
        'cardinal': 'ten',
        'ordinal': 'tenth',
        'number-integer': '1',
        'number-sequence': '1 2 3',
        'flight-number' : 'LH4234',
        'unit-area': 'ten square feet',
        'unit-currency': '5 dollars',
        'unit-length': 'ten meters',
        'unit-speed': '5 km/h',
        'unit-volume': '2 liters',
        'unit-weight': '5 kilos',
        'unit-information': '5 megabytes',
        'percentage': '10 percent',
        'temperature': '25 degrees',
        'duration': '5 days',
        'age': '1 year old',
        'currency-name': 'euros',
        'unit-area-name': 'suqare meters',
        'unit-length-name': 'meters',
        'unit-speed-name': 'kilometer per hour',
        'unit-volume-name': 'cubic meters',
        'unit-weight-name': 'kilograms',
        'unit-information-name': 'megabytes',
        'address': '1600 Amphitheatre Pkwy, Mountain View, CA 94043',
        'zip-code': '94122',
        'geo-capital': 'Rome',
        'geo-country': 'Denmark',
        'geo-country-code': 'US',
        'geo-city': 'Tokyo',
        'geo-state': 'Scotland',
        'place-attraction': 'Golden Gate Bridge',
        'airport': 'SFO',
        'location': '1600 Amphitheatre Pkwy, Mountain View, CA 94043',
        'email': 'test@example.com',
        'phone-number': '+11234567890',
        'given-name': 'Joe',
        'last-name': 'Smith',
        'music-artist': 'Beatles',
        'music-genre': 'Jazz',
        'color': 'Blue',
        'language': 'Japanese',
        'any': 'flower',
        'url': 'google.com'
    }
    for intent in intents:
        entities_used = {entity.display_name 
            for entity in intent.parameters}

        for entity in entities:
            if entity.display_name in entities_used and entity.display_name not in entities_name_to_value:
                entities_name_to_value[entity.display_name] = np.random.choice(
                  np.random.choice(entity.entities).synonyms, replace=False)

    intent_training_phrases = defaultdict(list)
    for intent in intents:
        for training_phrase in intent.training_phrases:

            parts = [
              entities_name_to_value[part.alias] 
              if part.alias in  entities_name_to_value else part.text
              for part in training_phrase.parts
          ]
            intent_training_phrases[intent.display_name].append(
              "".join(parts))
      # Remove intents with no training phrases
        if not intent_training_phrases[intent.display_name]:
            del intent_training_phrases[intent.display_name]
    return intent_training_phrases

In [49]:
intent_data = fetch_intents_training_phrases(credential_path, project_id)

In [60]:
len(intent_data.keys())

52

In [61]:
intent_data.keys()

dict_keys(['policy/loan_enquiry', 'cs/policy_pack/duplicate_policy_pack_with_change', 'credit_card/debit/incorrect', 'teleservicing/teleservicing/personal_update/mobile', 'policy/brochure/request', 'cs/policy_information/policy_inquiry/bonus_inquiry', 'pos/pos_request/documentation_required_for_pos_changes/credit_card_activation_process', 'cs/policy_information/policy_inquiry/surrender_value', 'pos/pos_request/documentation_required_for_pos_changes/nominee_change_process', 'loan/eligibility', 'cs/policy_information/policy_inquiry/due_date', 'escalation', 'policy/phishing_or_fraud_call', 'pos/pos_request/documentation_required_for_pos_changes/bank_details_process', 'mailroom/delivery_status/policy_pack_dispatch_status_or_clarification', 'teleservicing/teleservicing/personal_update/email', 'policy/bonus/eligibility', 'pos/pos_request/documentation_required_for_pos_changes/surrender_process', 'policy/personal_update/pan', 'policy/cancellation/form', 'cs/policy_pack/duplicate_policy_pack/s

In [59]:
label = []
training_phrase = []

In [62]:
for key in intent_data:
    label.extend([key]*len(intent_data[key]))
    training_phrase.extend(intent_data[key])

In [55]:
import pandas as pd

In [64]:
training_data = pd.DataFrame(data={'label':label, 'training_phrase':training_phrase})

In [66]:
training_data.to_csv("training_data_fetch_dialogflow.csv", index = False)

In [54]:
# intent_data['policy/loan_enquiry']

In [12]:
list(intents)

[name: "projects/intent-prediction-ohdl/agent/intents/05fbb988-4239-44f1-ae84-c86618f91fcb"
 display_name: "policy/loan_enquiry"
 priority: 500000
 messages {
   text {
   }
 },
 name: "projects/intent-prediction-ohdl/agent/intents/0c4456d7-24fc-44f3-90ad-a63214a7b5a9"
 display_name: "cs/policy_pack/duplicate_policy_pack_with_change"
 priority: 500000
 messages {
   text {
   }
 },
 name: "projects/intent-prediction-ohdl/agent/intents/0f28263a-222a-436e-bcb4-70ebb38294ae"
 display_name: "credit_card/debit/incorrect"
 priority: 500000
 parameters {
   name: "e2f75f59-c9dc-4943-b667-951df14b010e"
   display_name: "date-period"
   value: "$date-period"
   entity_type_display_name: "@sys.date-period"
   is_list: true
 }
 parameters {
   name: "fd4e163a-07b6-4f2c-84f9-b49f31799c3a"
   display_name: "date-period1"
   value: "$date-period1"
   entity_type_display_name: "@sys.date-period"
 }
 parameters {
   name: "f8f50f98-95d1-4ca8-92d0-085a07903852"
   display_name: "number"
   value: "$num

In [23]:
for intent in intents:
    print('=' * 20)
    print('Intent name: {}'.format(intent.name))
    print('Intent display_name: {}'.format(intent.display_name))
    all_intent_list.append(intent.display_name)
    print('Action: {}\n'.format(intent.action))
    print('Root followup intent: {}'.format(
        intent.root_followup_intent_name))
    print('Parent followup intent: {}\n'.format(
        intent.parent_followup_intent_name))

    print('Input contexts:')
    for input_context_name in intent.input_context_names:
        print('\tName: {}'.format(input_context_name))

    print('Output contexts:')
    for output_context in intent.output_contexts:
        print('\tName: {}'.format(output_context.name))

Intent name: projects/intent-prediction-ohdl/agent/intents/05fbb988-4239-44f1-ae84-c86618f91fcb
Intent display_name: policy/loan_enquiry
Action: 

Root followup intent: 
Parent followup intent: 

Input contexts:
Output contexts:
Intent name: projects/intent-prediction-ohdl/agent/intents/0c4456d7-24fc-44f3-90ad-a63214a7b5a9
Intent display_name: cs/policy_pack/duplicate_policy_pack_with_change
Action: 

Root followup intent: 
Parent followup intent: 

Input contexts:
Output contexts:
Intent name: projects/intent-prediction-ohdl/agent/intents/0f28263a-222a-436e-bcb4-70ebb38294ae
Intent display_name: credit_card/debit/incorrect
Action: 

Root followup intent: 
Parent followup intent: 

Input contexts:
Output contexts:
Intent name: projects/intent-prediction-ohdl/agent/intents/11180725-914c-4c3e-9266-c72a9bee3f30
Intent display_name: teleservicing/teleservicing/personal_update/mobile
Action: 

Root followup intent: 
Parent followup intent: 

Input contexts:
Output contexts:
Intent name: pro

In [11]:
len(all_intent_list)

52

# intent creation

In [12]:
def create_intent(
    intents_client,
    project_id,
    display_name,
    training_phrases_parts,
    message_texts
):
    """Create an intent of the given intent type."""
#     import dialogflow_v2 as dialogflow
#     intents_client = dialogflow.IntentsClient()

    parent = intents_client.project_agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part = dialogflow.types.Intent.TrainingPhrase.Part(
            text=training_phrases_part)
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part])
        training_phrases.append(training_phrase)

    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    intent = dialogflow.types.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message])

    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

In [41]:
train_file = pd.read_csv("dialogflow_filtered_data.csv")

In [42]:
train_file.head()

,intent_name,subticket
0,AA,Bt after few hour there also a deduction of rs...
1,AA,I have not enrolled into auto payment of my in...
2,AA,There has been deductions of months EMI from m...
3,AA,I have done as per instructions but you have n...
4,AA,i have suffiecient balance in my account for E...


In [43]:
train_file = train_file.drop_duplicates(subset="subticket").dropna()

In [44]:
train_file['seq_num'] = train_file.groupby('intent_name').cumcount()+1

In [45]:
train_file['phrase_count'] = train_file.groupby('intent_name')['subticket'].transform('count')

In [46]:
train_file['text_len'] = train_file['subticket'].str.len()

In [47]:
train_file['intent_name'].nunique()

52

In [48]:
train_file.head()

,intent_name,subticket,seq_num,phrase_count,text_len
0,AA,Bt after few hour there also a deduction of rs...,1,125,205
1,AA,I have not enrolled into auto payment of my in...,2,125,224
2,AA,There has been deductions of months EMI from m...,3,125,60
3,AA,I have done as per instructions but you have n...,4,125,143
4,AA,i have suffiecient balance in my account for E...,5,125,83


In [49]:
train_file[train_file['phrase_count']>2000].head()

,intent_name,subticket,seq_num,phrase_count,text_len
13370,BT,Re Max Life Insurance Policy Request to update...,1,3724,77
13371,BT,Dear Sir Madam I am Chandrani Chatterjee I am ...,2,3724,105
13372,BT,My credit card changed to new,3,3724,30
13373,BT,Mera card policy se de register jar diya jaye,4,3724,46
13374,BT,Payment successful home k Baad mujhe credit ca...,5,3724,119


In [50]:
train_file[train_file['phrase_count']>2000]['intent_name'].nunique()

4

In [51]:
train_file_filtered = train_file[(train_file['text_len']<768) & (train_file['seq_num']<2000)]

In [103]:
train_file_filtered.to_csv("filtered_train_data.csv", index = False)

In [52]:
train_file_filtered['intent_name'].nunique()

52

In [53]:
train_file_grouped = train_file_filtered.groupby('intent_name')['subticket'].apply(list).rename("text_grouped").reset_index()

In [54]:
train_file['intent_name'].nunique()

52

In [55]:
train_file_grouped.head()

,intent_name,text_grouped
0,AA,[Bt after few hour there also a deduction of r...
1,AB,[My credit card is active and sufficient limit...
2,AC,[Please tell me the Account value for my polic...
3,AD,[I want to say that please provide me the surr...
4,AE,[Dear Sir madam Kindly refer the subject polic...


In [56]:
for i,j in train_file_grouped.iterrows():
    intent_name = train_file_grouped['intent_name'][i]
    data = train_file_grouped['text_grouped'][i]
    if intent_name not in all_intent_list:
        create_intent(
            intents_client,
            project_id,
            intent_name,
            data,
            message_texts = None
        )

Intent created: name: "projects/intent-prediction-ohdl/agent/intents/e4cf21e5-d731-420e-8140-c01aed463e72"
display_name: "BT"
priority: 500000
messages {
  text {
  }
}

Intent created: name: "projects/intent-prediction-ohdl/agent/intents/f5536c3b-d330-4e14-9fdb-d8e40cee0ccc"
display_name: "BV"
priority: 500000
messages {
  text {
  }
}

Intent created: name: "projects/intent-prediction-ohdl/agent/intents/5292d175-ee1c-4875-83c7-39dd5343d756"
display_name: "BX"
priority: 500000
messages {
  text {
  }
}

Intent created: name: "projects/intent-prediction-ohdl/agent/intents/11180725-914c-4c3e-9266-c72a9bee3f30"
display_name: "BZ"
priority: 500000
messages {
  text {
  }
}



# intent prediction

In [57]:
session_client = dialogflow.SessionsClient(credentials=credentials)

In [58]:
session = session_client.session_path(project_id, "ankitnamdeo34")

In [59]:
DIALOGFLOW_LANGUAGE_CODE = 'en'

In [60]:
text_to_be_analyzed = "due date"

In [62]:
test_data = pd.read_csv("dialogflow_test_data.csv")

In [63]:
test_data.head()

,id,intent_name,subticket,clean_sentence
0,0,BX,regarding policy cancel t been informed me abo...,regarding policy cancel informed policy
1,1,BX,i would like to know the process to surrender ...,like know process surrender policies provide t...
2,2,BX,closing of policy,closing policy
3,3,BX,closing of policy no,closing policy no
4,4,BX,now please proceed with the cancellation of th...,proceed cancellation policy


In [81]:
test_data['text_len'] = test_data['subticket'].str.len()

In [84]:
test_data = test_data[test_data['text_len']< 256]

In [104]:
test_data.head()

,id,intent_name,subticket,clean_sentence,text_len,intent_predicted,intent_confidence,match
0,0,BX,regarding policy cancel t been informed me abo...,regarding policy cancel informed policy,59,BX,0.923329,1
1,1,BX,i would like to know the process to surrender ...,like know process surrender policies provide t...,169,AE,1.000000,0
2,2,BX,closing of policy,closing policy,18,BX,1.000000,1
3,3,BX,closing of policy no,closing policy no,21,BX,1.000000,1
4,4,BX,now please proceed with the cancellation of th...,proceed cancellation policy,55,BX,0.964973,1


In [85]:
def get_dialogflow_intent(row):
    text_to_be_analyzed = row['subticket']
    text_input = dialogflow.types.TextInput(text=text_to_be_analyzed, language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    response = session_client.detect_intent(session=session, query_input=query_input)
    intent_predicted = response.query_result.intent.display_name
    intent_confidnece = response.query_result.intent_detection_confidence
    return intent_predicted, intent_confidnece

In [86]:
test_data[['intent_predicted', 'intent_confidence']] = test_data.apply(get_dialogflow_intent, axis =1, result_type='expand')

D:\anaconda\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [87]:
test_data.head()

,id,intent_name,subticket,clean_sentence,text_len,intent_predicted,intent_confidence
0,0,BX,regarding policy cancel t been informed me abo...,regarding policy cancel informed policy,59,BX,0.923329
1,1,BX,i would like to know the process to surrender ...,like know process surrender policies provide t...,169,AE,1.000000
2,2,BX,closing of policy,closing policy,18,BX,1.000000
3,3,BX,closing of policy no,closing policy no,21,BX,1.000000
4,4,BX,now please proceed with the cancellation of th...,proceed cancellation policy,55,BX,0.964973


In [88]:
test_data.to_csv("predicted_res_dialogflow.csv", index = False)

In [89]:
test_data['intent_predicted'].nunique()

52

In [91]:
test_data['match'] = np.where(
    test_data['intent_predicted'] == test_data['intent_name'],
    1,
    0
)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [95]:
grouped_intent_res = test_data.groupby('intent_name')['match'].agg(['count','sum']).reset_index()

In [98]:
grouped_intent_res['accuracy'] = grouped_intent_res['sum']*100.0/grouped_intent_res['count']

In [100]:
grouped_intent_res.to_csv("accuracy_report.csv", index = False)

In [102]:
grouped_intent_res.sort_values("accuracy")

,intent_name,count,sum,accuracy
2,AC,24,0,0.000000
24,AZ,3,2,66.666667
17,AS,13,10,76.923077
25,BA,6,5,83.333333
43,BT,521,435,83.493282
45,BV,1238,1042,84.168013
23,AY,13,11,84.615385
47,BX,1156,984,85.121107
19,AU,61,57,93.442623
33,BJ,20,19,95.000000
